In [1]:
import pandas as pd
import numpy as np

In [2]:
url = '/Users/vigneshpadmanabhan/Downloads/vehicles.csv.zip'
df = pd.read_csv(url, low_memory=False)
city_mpg = df.city08
highway_mpg = df.highway08

.apply method applies a function element-wise to every value. 
- for eg NumPy array this function broadcasts the operation to the series.
- this could be considered as a codesmell because it repeats using the function if there are million rows.
- therefore doesn't use the vectorized approach instead takes the slow python approach.  

In [3]:
def gt20(val):
    return val > 20

In [4]:
%%timeit
city_mpg.apply(gt20)

5.67 ms ± 77.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
%%timeit
city_mpg.gt(20)

94.4 µs ± 1.31 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
make = df.make
make

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: object

In [7]:
top5 = make.value_counts().index[:5]
top5

Index(['Chevrolet', 'Ford', 'Dodge', 'GMC', 'Toyota'], dtype='object')

In [8]:
def generalize_top5(val):
    if val in top5:
        return val
    else:
        return 'Other'

In [9]:
make.apply(generalize_top5)

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: object

Faster method for doing this is by using "where" instead of generalize_top5

In [10]:
make.where(make.isin(top5),other = 'Other') #the where method uses a boolean array to check with the first condition and then applies the other incase not. 

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: object

In [11]:
%%timeit
make.apply(generalize_top5)

18 ms ± 2.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%%timeit
make.where(make.isin(top5),other = 'Other')

2 ms ± 44.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


the complement of "where" is "mask"

In [15]:
make.mask(make.isin(top5),other = 'Other') #Mask will check if the condition is false. therefore its good to provide ~in front of the condition to give it same outcome. 

0        Alfa Romeo
1           Ferrari
2             Other
3             Other
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: object

In [16]:
make.mask(~make.isin(top5),other = 'Other')

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: object

if - else with pandas

In [19]:
vc = make.value_counts()
vc

Chevrolet                      4003
Ford                           3371
Dodge                          2583
GMC                            2494
Toyota                         2071
                               ... 
Volga Associated Automobile       1
Panos                             1
Mahindra                          1
Excalibur Autos                   1
London Coach Co Inc               1
Name: make, Length: 136, dtype: int64

In [20]:
top5 = vc.index[:5]
top5

Index(['Chevrolet', 'Ford', 'Dodge', 'GMC', 'Toyota'], dtype='object')

In [22]:
top10 = vc.index[:10]
top10

Index(['Chevrolet', 'Ford', 'Dodge', 'GMC', 'Toyota', 'BMW', 'Mercedes-Benz',
       'Nissan', 'Volkswagen', 'Mitsubishi'],
      dtype='object')

In [23]:
def generalize(val):
    if val in top5:
        return val
    elif val in top10:
        return 'top10'
    else:
        return 'Other'

In [24]:
vc.apply(generalize)

Chevrolet                      Other
Ford                           Other
Dodge                          Other
GMC                            Other
Toyota                         Other
                               ...  
Volga Associated Automobile    Other
Panos                          Other
Mahindra                       Other
Excalibur Autos                Other
London Coach Co Inc            Other
Name: make, Length: 136, dtype: object

In [26]:
z = (make
.where(make.isin(top5), other = 'top10')
.where(make.isin(top10), other = 'Other')
)
z

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: object

In [29]:
z.unique()

array(['Other', 'Dodge', 'Toyota', 'top10', 'Chevrolet', 'Ford', 'GMC'],
      dtype=object)

In [34]:
# alternate to where using select from numpy
pd.Series(np.select([make.isin(top5), make.isin(top10)],[make,'Top10'],'Other'))


0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Length: 41144, dtype: object

Missing values

In [39]:
cyl = df.cylinders
cyl.unique()

array([ 4., 12.,  8.,  6.,  5., 10.,  2.,  3., nan, 16.])

In [38]:
(cyl
.isna()
.sum())

206

In [43]:
# using make with the corresponding missing to give us better insight. 
missing = cyl.isna()
make.loc[missing]

7138     Nissan
7139     Toyota
8143     Toyota
8144       Ford
8146       Ford
          ...  
34563     Tesla
34564     Tesla
34565     Tesla
34566     Tesla
34567     Tesla
Name: make, Length: 206, dtype: object

In Pandas
 - both series and dataframe have an index
 - for index operations we use ([]) to select values from a series or a dataframe
 - .loc represnts part of the index whether its a series or a dataframe 

In [44]:
# filling in missing values

cyl[cyl.isna()]

7138    NaN
7139    NaN
8143    NaN
8144    NaN
8146    NaN
         ..
34563   NaN
34564   NaN
34565   NaN
34566   NaN
34567   NaN
Name: cylinders, Length: 206, dtype: float64

In [45]:
cyl.fillna(0).loc[7136:7141]

7136    6.0
7137    6.0
7138    0.0
7139    0.0
7140    6.0
7141    6.0
Name: cylinders, dtype: float64

Interpolate data

In [46]:
temp = pd.Series([32,40,None,42,39,32])
temp

0    32.0
1    40.0
2     NaN
3    42.0
4    39.0
5    32.0
dtype: float64

In [47]:
temp.interpolate()

0    32.0
1    40.0
2    41.0
3    42.0
4    39.0
5    32.0
dtype: float64

Clipping data

In [52]:
city_mpg.loc[:466]

0      19
1       9
2      23
3      10
4      17
       ..
462    15
463    15
464    15
465    13
466    13
Name: city08, Length: 467, dtype: int64

In [55]:
(city_mpg
.loc[0:466]
.clip(lower = city_mpg.quantile(0.05),
      upper = city_mpg.quantile(.95))
.astype('int32')
)

0      19
1      11
2      23
3      11
4      17
       ..
462    15
463    15
464    15
465    13
466    13
Name: city08, Length: 467, dtype: int32

Sort_values

In [56]:
temp.sort_values()

0    32.0
5    32.0
4    39.0
1    40.0
3    42.0
2     NaN
dtype: float64

In [57]:
city_mpg.sort_values()

7901       6
34557      6
37161      6
21060      6
35887      6
        ... 
34563    138
34564    140
32599    150
31256    150
33423    150
Name: city08, Length: 41144, dtype: int64

In [59]:
(city_mpg.sort_values() + highway_mpg) / 2

0        22.0
1        11.5
2        28.0
3        11.0
4        20.0
         ... 
41139    22.5
41140    24.0
41141    21.0
41142    21.0
41143    18.5
Length: 41144, dtype: float64

Sort index of a series

In [60]:
city_mpg.sort_values().sort_index()

0        19
1         9
2        23
3        10
4        17
         ..
41139    19
41140    20
41141    18
41142    18
41143    16
Name: city08, Length: 41144, dtype: int64

In [61]:
city_mpg

0        19
1         9
2        23
3        10
4        17
         ..
41139    19
41140    20
41141    18
41142    18
41143    16
Name: city08, Length: 41144, dtype: int64

Dropping duplicates

In [65]:
city_mpg.drop_duplicates() # default is keep first

0         19
1          9
2         23
3         10
4         17
        ... 
34364    127
34409    114
34564    140
34565    115
34566    104
Name: city08, Length: 105, dtype: int64

In [63]:
city_mpg.drop_duplicates(keep='last')

8147     84
10329    87
12922    45
23028    59
23029    79
         ..
41138    21
41139    19
41140    20
41142    18
41143    16
Name: city08, Length: 105, dtype: int64

In [64]:
city_mpg.drop_duplicates(keep = False)

8147      84
23028     59
23029     79
24471    107
25699     60
25953     93
32740    131
32842    125
34173    123
34364    127
34409    114
34564    140
34565    115
34566    104
Name: city08, dtype: int64

Ranking dataset

In [67]:
city_mpg.rank()

0        27060.5
1          235.5
2        35830.0
3          607.5
4        19484.0
          ...   
41139    27060.5
41140    29719.5
41141    23528.0
41142    23528.0
41143    15479.0
Name: city08, Length: 41144, dtype: float64

In [72]:
city_mpg.rank(method = 'min') # puts equal value in the same rank

0        25555.0
1          136.0
2        35119.0
3          336.0
4        17467.0
          ...   
41139    25555.0
41140    28567.0
41141    21502.0
41142    21502.0
41143    13492.0
Name: city08, Length: 41144, dtype: float64

In [73]:
city_mpg.rank(method = 'dense') #to not skip any position

0        14.0
1         4.0
2        18.0
3         5.0
4        12.0
         ... 
41139    14.0
41140    15.0
41141    13.0
41142    13.0
41143    11.0
Name: city08, Length: 41144, dtype: float64

In [76]:
new_temp = temp.interpolate()
new_temp

0    32.0
1    40.0
2    41.0
3    42.0
4    39.0
5    32.0
dtype: float64

In [77]:
new_temp.rank()

0    1.5
1    4.0
2    5.0
3    6.0
4    3.0
5    1.5
dtype: float64

In [78]:
new_temp.rank(method='min')

0    1.0
1    4.0
2    5.0
3    6.0
4    3.0
5    1.0
dtype: float64

In [79]:
new_temp.rank(method='dense')

0    1.0
1    3.0
2    4.0
3    5.0
4    2.0
5    1.0
dtype: float64

Replace method

In [80]:
make.replace('Subaru','waste')

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4             waste
            ...    
41139         waste
41140         waste
41141         waste
41142         waste
41143         waste
Name: make, Length: 41144, dtype: object

In [83]:
new_temp = new_temp.astype('int32')
new_temp

0    32
1    40
2    41
3    42
4    39
5    32
dtype: int32

In [84]:
new_temp.replace([42,32],[44,36])

0    36
1    40
2    41
3    44
4    39
5    36
dtype: int32